In [39]:
import os
import openpyxl

def isValidScorePresent(sheet,r,c):
    if sheet.cell(row = r + 1, column=c).value != 'NT' \
            and sheet.cell(row = r + 1, column=c).value != 'I' \
            and sheet.cell(row = r + 1, column=c).value != 'XX' \
            and sheet.cell(row = r + 1, column=c).value != None:
        return True
    else:
        return False

def get_value(sheet,u,i):
    return sheet.cell(row = u+1,column = i).value

print("Collaborative Filtering Item-Based Algorithm for Course recommendations and Grade Prediction")
wb= openpyxl.load_workbook('student.xlsx')


sheetname = wb.get_sheet_names()

print("\nStudent's score matrix(Student ID in first column)\n")

sheet=wb.get_sheet_by_name(sheetname[1])
# print(sheet)
for i in range(2,sheet.max_row+1,1):
    for j in range(1,sheet.max_column+1,1):
        if sheet.cell(row=i,column=j).value != None:
            if j == 1:
                if int(sheet.cell(row=i,column=j).value/10)==0:
                    print(sheet.cell(row=i, column=j).value, end="  | ")
                else:
                    print(sheet.cell(row=i,column=j).value,end=" |  ")
            else:
                print(sheet.cell(row=i,column=j).value,end="  ")
            if sheet.cell(row=i, column=j).value != 'I' and sheet.cell(row=i, column=j).value != 'XX' and sheet.cell(row=i, column=j).value != 'NT':
                if int(sheet.cell(row=i,column=j).value/10) == 0:
                    print(end=" ")
            if sheet.cell(row=i,column=j).value=='I':
                print(end=" ")
        else:
            print("-",end="   ")
    print()
studentID = int(input("\nEnter student Roll NO : "))
# print(studentID);
coursesList = []
print("\nAvailable courses for you are as below")
for i in range (2,sheet.max_column+1):
    if (i-2)%9 == 0:
        print()
    print(sheet.cell(row=1,column=i).value,end=", ")
    coursesList.append(sheet.cell(row=1,column=i).value)
print("\n")
while True:
    try:
        c = input("\nEnter course: ")
        course = coursesList.index(c)+1
        break
    except ValueError:
        print("Please choose a course from given list")
        
###---------item based-------
#similarity_between_courses b/w u,courses
# calculates average score of each  course for all students
def compute_average_score(sheet,student):    # calculate avg grades for student
    sum = 0
    cnt = 0
    #iterate over courses and get score value for each score for given student and calculate total sum
    for i in range (2,sheet.max_column+1):
        if isValidScorePresent(sheet,student,i):
            sum = sum + int(get_value(sheet,student,i))
            cnt = cnt + 1
    if cnt != 0:
        return round(float(sum/cnt),3) #compute average of  score for gievn student= total sum of all courses score/total no of courses
    else:
        return 0

#compute similarity score between courses
def similarity_between_courses(c1, c2, sheet):
    sum1 = 0
    sum2 = 0
    sum3 = 0
    # avgV = compute_average_score(sheet, v)
    # print(" avg ",avgU,avgV,v)
    
    for i in range(1,sheet.max_row):
        if isValidScorePresent(sheet,i,c1) and isValidScorePresent(sheet,i,c2):
            avg = compute_average_score(sheet,i)
            # print(i,avg)
            sum1 = sum1 + (get_value(sheet,i,c1) - avg) * (get_value(sheet,i,c2) - avg)
            sum2 = sum2 + (get_value(sheet,i,c1) - avg) * (get_value(sheet,i,c1) - avg)
            sum3 = sum3 + (get_value(sheet,i,c2) - avg) * (get_value(sheet,i,c2) - avg)
    if sum2!=0 and sum3!=0:
        return round(float(sum1/((sum2*sum3)**(1/2))),3)
    else:
        return 0

#compute similarity score for each courses ad stores in similar_item_val list
similar_item_val = []
similar_coursesName_value = []
simi_item = []
highest_similarityBetweencourses = []


avgU = compute_average_score(sheet, studentID)

print("\nBelow are recommended courses based on Item based collaborative filtering:")
for i in range(2,sheet.max_column+1):
        if i!=course:
            sim = similarity_between_courses(i,course,sheet)
            course1Name =sheet.cell(row=1,column=course+1).value
            course2Name =sheet.cell(row=1,column=i+1).value
            similar_item_val.append([course1Name,course2Name,sim])
#             print(sheet.cell(row=1,column=course).value,end=", ")
#             print(sheet.cell(row=1,column=i).value,end=", ")
#             print(sim)
        if sim > 0:
            courseName1 =sheet.cell(row=1,column=course+1).value
            courseName2 =sheet.cell(row=1,column=i+1).value
            simi_item.append([course,i,sim])
            highest_similarityBetweencourses.append([courseName1,courseName2,sim])
            similar_coursesName_value.append(courseName2)
         
print("\n")
print("Rank ", end= "|")      
print("CourseNames ", end= " ")  
for i in range(0,len(similar_coursesName_value)-1):
  print("\n")
  print(i,end="    |")
  print(similar_coursesName_value[i],end="")
  
  
# print(similar_item_val)
#print(simi_item)
#print("Below is the similarity between courses" )
#print( highest_similarityBetweencourses)



#predicting grade
sum1 = 0
sum2 = 0
for i in range(0,len(simi_item)):
    if isValidScorePresent(sheet,studentID,int(simi_item[i][1])):
        sum1 = sum1 + (simi_item[i][2] * get_value(sheet,studentID,simi_item[i][1]))
        sum2 = sum2 + simi_item[i][2]
if sum2 != 0:
    print("\n")
    print("\nPredicted Grade for studentID "+str(studentID)+" in subject "+str(c)+" : "+str(round(float(sum1/sum2),3)))

Collaborative Filtering Item-Based Algorithm for Course recommendations and Grade Prediction

Student's score matrix(Student ID in first column)

1  |  4   5   6   6   5   7   4   6   6   8   5   NT  5   4   6   7   5   7   3   7   7   4   8   9   5   5   
2  |  5   7   7   7   6   8   5   0   0   8   5   NT  3   4   6   6   5   5   4   6   6   4   4   7   5   6   
3  |  7   4   6   6   5   8   7   7   7   8   7   NT  6   6   8   7   8   7   6   6   6   5   8   8   4   6   
4  |  7   7   9   9   7   8   10  9   9   9   7   NT  8   8   5   8   8   8   6   8   8   9   10  9   7   7   
5  |  4   4   6   6   3   7   5   5   6   8   6   NT  4   5   4   5   4   5   3   6   6   5   7   5   0   4   
6  |  9   8   7   7   8   9   7   8   9   8   8   NT  7   7   6   8   7   5   7   7   7   8   8   10  6   7   
7  |  8   5   6   6   7   8   8   7   7   8   7   NT  5   8   8   8   7   8   6   8   8   8   10  9   7   7   
8  |  I   XX  I   I   I   I   0   0   0   I   I   NT  I   -   -   -   -   -  

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: DeprecationWarning: Call to deprecated function get_sheet_names (Use wb.sheetnames).
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: DeprecationWarning: Call to deprecated function get_sheet_by_name (Use wb[sheetname]).


7   6   6   10  4   8   6   9   
12 |  6   9   7   7   5   10  5   7   6   8   6   NT  7   5   6   7   4   9   7   7   7   9   4   6   6   9   
13 |  5   8   7   7   6   8   0   5   5   8   5   NT  3   4   4   5   3   5   3   6   6   5   0   7   4   6   
14 |  4   7   8   8   5   8   4   5   6   8   7   NT  7   4   5   6   4   3   3   6   6   6   0   0   4   6   
15 |  9   7   7   7   7   10  10  8   8   9   7   NT  7   6   5   9   9   8   6   8   8   7   8   6   4   8   
16 |  5   9   6   6   6   9   4   3   6   8   4   NT  6   4   4   9   5   8   5   8   8   7   6   9   5   7   
17 |  8   9   9   9   7   9   9   8   10  9   10  NT  8   10  10  9   10  9   6   7   7   7   7   10  7   8   
18 |  7   9   8   8   8   10  5   8   8   8   6   NT  7   7   6   8   9   7   8   9   9   7   9   10  8   8   
19 |  7   6   9   9   7   9   10  10  8   9   8   NT  6   7   4   9   6   8   6   8   8   7   8   7   6   7   
20 |  5   6   7   7   7   9   10  8   9   9   6   NT  6   8   5   8   7   7   5